In [6]:
import os
from pathlib import Path
from dotenv import load_dotenv

ENV_FILE = f'../.env'
load_dotenv(dotenv_path=Path(ENV_FILE), override=True)

username=os.getenv('username')
password=os.getenv('password')

host='https://api.worldquantbrain.com'

In [7]:
import requests
from requests.auth import HTTPBasicAuth

session = requests.Session()
session.auth = HTTPBasicAuth(username, password)
response = session.post(f"{host}/authentication")

print(response.status_code)
print(response.json())

201
{'user': {'id': 'AX49862'}, 'token': {'expiry': 14400.0}, 'permissions': ['TUTORIAL']}


In [ ]:
simulation_data = {
    'type':'REGULAR',
    'settings':{
        'instrumentType':'EQUITY',
        'region':'USA',
        'universe':'TOP3000',
        'delay':1,
        'decay':0,
        'neutralization':'INDUSTRY',
        'truncation':0.08,
        'pasteurization':'ON',
        'unitHandling':'VERIFY',
        'nanHandling':'OFF',
        'language':'FASTEXPR',
        'visualization':False
    },
    'regular': 'liabilities/assets'
}

from time import sleep

sim_resp = session.post(
    'https://api.worldquantbrain.com/simulations',
    json = simulation_data,
)
sim_progress_url = sim_resp.headers['Location']

while True:
    sim_progress_resp = session.get(sim_progress_url)
    retry_after_sec = float(sim_progress_resp.headers.get("retry-After",0))
    if retry_after_sec == 0 :
        break
    sleep(retry_after_sec)

response = sim_progress_resp.json()
print(response)

alpha_id = response["alpha"]
print(f"Alpha ID: {alpha_id}")

: 

In [ ]:
def submit_alpha(alpha_id):
    url = f"https://api.worldquantbrain.com/alphas/{alpha_id}/submit"
    submit_response = session.post(url)
    while True:
        submit_response = session.get(sim_progress_url)
        retry_after_sec = float(submit_response.headers.get("retry-After",0))
        if retry_after_sec == 0:
            print("Alpha submitted successfully.")
            print(submit_response.status_code)
            print(submit_response.json())
            break
        sleep(retry_after_sec)
        # if 'retry-after' in submit_response.headers:
        #     retry_after_sec = float(submit_response.headers['retry-after'])
        #     sleep(retry_after_sec)
        #     submit_response = session.get(url)
        #     print(submit_response.status_code)
        #     print(submit_response.json())
        # else:
        #     break
# submit_alpha('XAaQR0x')

Alpha submitted successfully.
200
{'id': '3crPxZ9Eb4gWbUF8NP4vTz7', 'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'INDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'OFF', 'maxTrade': 'OFF', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'liabilities/assets', 'status': 'COMPLETE', 'alpha': 'XAaQR0x'}


In [11]:
def get_data_fields(session, search_scope, dataset_id = '', search =''):
    import pandas as pd
    instrumentType = search_scope['instrumentType']
    region = search_scope['region']
    delay = str(search_scope['delay'])
    universe = search_scope['universe']
    if len(search) == 0:
        url_template = f"{host}/data-fields?" +\
            f"&instrumentType={instrumentType}" +\
            f"&region={region}&delay={delay}&universe={universe}" +\
            f"&dataset.id={dataset_id}&limit=50" +\
            "&offset={x}"
        url = url_template.format(x=0)
        count = session.get(url).json()['count']
    else:
        url_template = f"{host}/data-fields?" +\
            f"&instrumentType={instrumentType}" +\
            f"&region={region}&delay={delay}&universe={universe}" +\
            f"&dataset.id={dataset_id}&limit=50" +\
            f"&search={search}" +\
            "&offset={x}"
        count = 100

    data_fields = []
    for x in range(0, count, 50):
        url = url_template.format(x=x)
        response = session.get(url)
        data = response.json()
        print(data)
        data_fields.append(data['results'])
    data_fields_flat = [item for sublist in data_fields for item in sublist]
    data_fields_df = pd.DataFrame(data_fields_flat)
    return data_fields_df

In [13]:
search_scope = {
    'instrumentType': 'EQUITY',
    'region': 'USA',
    'delay': 1,
    'universe': 'TOP3000'
}
fd6_fields = get_data_fields(session, search_scope, "fundamental6")
fd6_matrix = fd6_fields[fd6_fields['type'] == 'MATRIX']
fd6_matrix.head()

{'count': 886, 'results': [{'id': 'assets', 'description': 'Assets - Total', 'dataset': {'id': 'fundamental6', 'name': 'Company Fundamental Data for Equity'}, 'category': {'id': 'fundamental', 'name': 'Fundamental'}, 'subcategory': {'id': 'fundamental-fundamental-data', 'name': 'Fundamental Data'}, 'region': 'USA', 'delay': 1, 'universe': 'TOP3000', 'type': 'MATRIX', 'coverage': 0.9524, 'userCount': 27516, 'alphaCount': 97405, 'themes': []}, {'id': 'assets_curr', 'description': 'Current Assets - Total', 'dataset': {'id': 'fundamental6', 'name': 'Company Fundamental Data for Equity'}, 'category': {'id': 'fundamental', 'name': 'Fundamental'}, 'subcategory': {'id': 'fundamental-fundamental-data', 'name': 'Fundamental Data'}, 'region': 'USA', 'delay': 1, 'universe': 'TOP3000', 'type': 'MATRIX', 'coverage': 0.7655, 'userCount': 2568, 'alphaCount': 12954, 'themes': []}, {'id': 'bookvalue_ps', 'description': 'Book Value Per Share', 'dataset': {'id': 'fundamental6', 'name': 'Company Fundamenta

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,userCount,alphaCount,themes
0,assets,Assets - Total,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.9524,27516,97405,[]
1,assets_curr,Current Assets - Total,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.7655,2568,12954,[]
2,bookvalue_ps,Book Value Per Share,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.9754,2059,8822,[]
3,capex,Capital Expenditures,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.9646,8449,21865,[]
4,cash,Cash,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.7529,1926,10535,[]


In [14]:
fd6_matrix['id'].values

array(['assets', 'assets_curr', 'bookvalue_ps', 'capex', 'cash',
       'cash_st', 'cashflow', 'cashflow_dividends', 'cashflow_fin',
       'cashflow_invst', 'cashflow_op', 'cogs', 'current_ratio', 'debt',
       'debt_lt', 'debt_st', 'depre_amort', 'ebit', 'ebitda', 'employee',
       'enterprise_value', 'eps', 'equity', 'fnd6_acdo', 'fnd6_acodo',
       'fnd6_acox', 'fnd6_acqgdwl', 'fnd6_acqintan',
       'fnd6_adesinda_curcd', 'fnd6_aldo', 'fnd6_am', 'fnd6_aodo',
       'fnd6_aox', 'fnd6_aqc', 'fnd6_aqi', 'fnd6_aqs', 'fnd6_beta',
       'fnd6_capxv', 'fnd6_ceql', 'fnd6_ch', 'fnd6_ci', 'fnd6_cibegni',
       'fnd6_cicurr', 'fnd6_cidergl', 'fnd6_cik', 'fnd6_cimii',
       'fnd6_ciother', 'fnd6_cipen', 'fnd6_cisecgl', 'fnd6_citotal',
       'fnd6_city', 'fnd6_cld2', 'fnd6_cld3', 'fnd6_cld4', 'fnd6_cld5',
       'fnd6_cptmfmq_actq', 'fnd6_cptmfmq_atq', 'fnd6_cptmfmq_ceqq',
       'fnd6_cptmfmq_dlttq', 'fnd6_cptmfmq_dpq', 'fnd6_cptmfmq_lctq',
       'fnd6_cptmfmq_oibdpq', 'fnd6_cptmfmq_o

In [18]:
values = fd6_matrix['id'].values.tolist()
len(values)

574